In [ ]:
!pip install pycocotools

In [ ]:
import os
from pycocotools.coco import COCO
import json

In [ ]:
import os
import json
from pycocotools.coco import COCO
from urllib.request import urlopen
from PIL import Image
import shutil

In [ ]:
desired_classes = [
    'banana',
    'bench',
    'bicycle',
    'bird',
    'bus',
    'car',
    'cat',
    'cow',
    'dog',
    'elephant',
    'fire hydrant',
    'horse',
    'person',
    'potted plant',
    'skateboard',
    'sports ball',
    'stop sign',
    'traffic light',
    'train',
    'truck',
    'umbrella'
]


In [ ]:
!wget http://images.cocodataset.org/zips/train2017.zip

--2023-10-16 15:31:28--  http://images.cocodataset.org/zips/train2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 54.231.200.225, 52.217.235.97, 54.231.224.1, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|54.231.200.225|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19336861798 (18G) [application/zip]
Saving to: ‘train2017.zip’

train2017.zip       100%[===================>]  18.01G  91.2MB/s    in 4m 20s  

2023-10-16 15:35:48 (71.0 MB/s) - ‘train2017.zip’ saved [19336861798/19336861798]



In [ ]:
!wget http://images.cocodataset.org/zips/val2017.zip

--2023-10-16 15:35:48--  http://images.cocodataset.org/zips/val2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.37.225, 54.231.172.81, 52.217.77.156, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.37.225|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 815585330 (778M) [application/zip]
Saving to: ‘val2017.zip’

val2017.zip         100%[===================>] 777.80M  78.6MB/s    in 10s     

2023-10-16 15:35:58 (76.2 MB/s) - ‘val2017.zip’ saved [815585330/815585330]



In [ ]:
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip

--2023-10-16 15:35:58--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.32.81, 54.231.140.113, 54.231.201.193, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.32.81|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘annotations_trainval2017.zip’

annotations_trainva 100%[===================>] 241.19M  75.0MB/s    in 3.2s    

2023-10-16 15:36:02 (75.0 MB/s) - ‘annotations_trainval2017.zip’ saved [252907541/252907541]



In [ ]:
!unzip train2017.zip
!unzip val2017.zip
!unzip annotations_trainval2017.zip

In [ ]:
desired_classes = [
    'banana', 'bench', 'bicycle', 'bird', 'bus', 'car', 'cat', 'cow', 'dog',
    'elephant', 'fire hydrant', 'horse', 'person', 'potted plant', 'skateboard',
    'sports ball', 'stop sign', 'traffic light', 'train', 'truck', 'umbrella'
]

dataDir = '/content/dataDir'
dataType = 'train2017'
annFile = f'{dataDir}/annotations/instances_{dataType}.json'

coco = COCO(annFile)

cat_id_to_name = {cat['id']: cat['name'] for cat in coco.dataset['categories']}

filtered_images_folder = '/content/Images'
os.makedirs(filtered_images_folder, exist_ok=True)

filtered_image_ids = set()
filtered_annotations = []
desired_num_images = 10000
num_collected_images = 0

for cat_name in desired_classes:
    cat_id = next(cat_id for cat_id, cat_name_in_coco in cat_id_to_name.items() if cat_name_in_coco == cat_name)
    ann_ids = coco.getAnnIds(catIds=cat_id)
    filtered_annotations.extend(coco.loadAnns(ann_ids))
    filtered_image_ids.update(set(ann['image_id'] for ann in filtered_annotations))

    num_collected_images = len(filtered_image_ids)
    if num_collected_images >= desired_num_images:
        break

for image_id in filtered_image_ids:
    img_info = coco.loadImgs(image_id)[0]
    img_url = img_info['coco_url']
    img_name = os.path.basename(img_url)
    img_path = os.path.join(filtered_images_folder, img_name)

    with urlopen(img_url) as response, open(img_path, 'wb') as out_file:
        shutil.copyfileobj(response, out_file)


filtered_annotations_path = '/content/filtered_annotations.json'
with open(filtered_annotations_path, 'w') as annotations_file:
    json.dump(filtered_annotations, annotations_file)


loading annotations into memory...
Done (t=14.65s)
creating index...
index created!


In [ ]:
# prompt: UPLOAD TO DRIVE

!zip -r -q /content/Images.zip /content/Images
!gdrive upload /content/Images.zip


/bin/bash: line 1: gdrive: command not found


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
shutil.copy("/content/filtered_annotations.json","/content/drive/MyDrive/DATASET_FINAL_ANNOTATIONS+IMAGES")


'/content/drive/MyDrive/DATASET_FINAL_ANNOTATIONS+IMAGES/filtered_annotations.json'

In [ ]:
shutil.copy("/content/Images.zip","/content/drive/MyDrive/DATASET_FINAL_ANNOTATIONS+IMAGES")


'/content/drive/MyDrive/DATASET_FINAL_ANNOTATIONS+IMAGES/Images.zip'